In [209]:

# remember to load the environment first by running `conda activate colombia_analysis`
#import all the handy dandy libraries
import sys
import numpy as np
import pandas as pd
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
from matplotlib import gridspec
%matplotlib inline

#get useful tools from augur
from augur.utils import json_to_tree

#import baltic
import imp
bt = imp.load_source('baltic','./baltic.py')

#datetime libraries
import datetime
from datetime import timedelta

In [267]:
#some useful functions

def decimalDateToCalendarDate(decimalDate):
    """ Converts decimal dates to YYYY-MM-DD format"""
    year = int(decimalDate)
    dt_format_year = datetime.datetime(year, 1, 1)
    remainder = decimalDate - year
    calendar_date = dt_format_year + timedelta(seconds=(dt_format_year.replace(year=dt_format_year.year + 1) - dt_format_year).total_seconds() * remainder)
    return calendar_date.strftime('%Y-%m-%d')


def timestampToDecimalDate(sampling_date):
    """ This function takes in a Datetime Timestamp object and retuns the decimal date format of the date.
        Note that the decimal date will count all time UP TO the specified calendar date. For example, if my date is Jan 3,
        then the decimal fraction will represent two days worth of seconds (all the seconds of Jan 1 and Jan 2).
        
        Also note that this function requires the timestamp to be in Datetime format, which is DIFFERENT than pandas'
        timestamp format. Therefore, if dates are coming from a pandas df, you'll likely need to convert them with x.to_pydatetime()
        before you can pass them to thus function."""
    
    year =  sampling_date.year
    beginning_of_year = dt.datetime(year,1,1) #year, month, day, so year 1,1 == beginning of year
    end_of_year = dt.datetime(year+1,1,1) #next year 1,1 is a full year away from beginning of sampling year
    #return fraction of the full year (in seconds) that occurs between jan 1st and your sampling date
    return year + ((sampling_date - beginning_of_year).total_seconds() / ((end_of_year - beginning_of_year).total_seconds()))

In [214]:
#load tree in as BALTIC tree object
tree = bt.loadJSON(json_object="./humboldt_ancestors_210308.json",json_translation={'name':'name','absoluteTime':'num_date'},verbose=False,sort=True,stats=True)

print(tree[0])


Tree height: 1.192331
Tree length: 235.684820
multitype tree
annotations present

Numbers of objects in tree: 1690 (398 nodes and 1292 leaves)



In [268]:
#there are a few samples that represent Humboldt County but were collected by other groups. Want to make sure they're captured.
humboldt_samples_originating_labs = ["Humboldt County Public Health Laboratory","Chiu Laboratory, University of California, San Francisco", "California Department of Public Health"]

#and then parse through the tree and grab collection dates for any sample that was collected from Humboldt.
sample_collection_dates = {}
for k in tree[0].Objects: #actual BALTIC tree object is the first element of the tuple after import.
    if k.branchType == "leaf" and k.traits["originating_lab"] in humboldt_samples_originating_labs:
        try:
            if k.traits["location"] == "Humboldt County":
                sample_collection_dates[k.name] = [decimalDateToCalendarDate(k.absoluteTime)]
            
        except KeyError: #not every sample has a "location" value. But all Humboldt ones will.
            pass

In [269]:
#take this dict and put it into pandas df format (and make date pandas compatible)
seq_collection_dates_df = pd.DataFrame.from_dict(sample_collection_dates, orient='index', columns=['date'])

In [271]:
seq_collection_dates_df

,date
P20-0911,2020-04-06
P20-3252,2020-05-27
P20-0508,2020-03-27
P20-0505,2020-03-27
P20-1130,2020-04-13
...,...
P21-03864,2021-01-26
P21-03869,2021-01-25
P21-03627,2021-01-25
P21-04136,2021-01-27


In [309]:
#then, for the df we have with sequenced sample collection dates, get numbers of sequences collected per day.
n_genomes_per_day = pd.DataFrame(seq_collection_dates_df["date"].value_counts().sort_index())
name_fixed_df = n_genomes_per_day.rename(columns={"date":"count"})
name_fixed_df.to_csv("./n_sequenced_genomes_per_day.csv", index_label = "date")

######## If Joe wants this series filled in with days that had 0 collected sequences, then just do what's commented out below
######## which basically makes a df filled with 0's for all days in the index df, and then adds our actual counts to it.

### make a df that spans every date from the first collection date of a sequenced sample to the last collection date of a sequenced sample.
#date_index = pd.date_range(seq_collection_dates_df["date"].min(), seq_collection_dates_df["date"].max(), freq='d')
#empty_curve = case_counts_per_day.reindex(day_index).fillna(0)
#epi_curve = empty_curve.add(case_counts_per_day).fillna(0)